# Main function and Sub Functions
Use for finding the Soft and Hard Bin counts when having Lot and Wafer Number and EG to Bin Counts. This function will helps you automatically finds the followings :
1. designator
2. scribe_family
3. Bin_1 Counts
4. Bin_2 Counts
5. Bin_7 Counts
6. Distinct Softbin Values
7. Yield
8. Yield_loss

(For confidentail reason the numbers displayed are fake and some outputs are deleted)


In [0]:
import pandas as pd
from collections import defaultdict
# Read the first CSV file and data engineering
def eng_dic(input_path):
  res = defaultdict(list)
  df = pd.read_csv(input_path)
  df.rename(columns={df.columns[0]: 'Lot', df.columns[1]: 'Wafer'}, inplace=True)
  for i in range(len(df)):
    res[str(df.loc[(i), 'Lot'])].append(df.loc[(i), 'Wafer']) 
  return res

# sub function for query
def Lot_wafer_to_family(lot, wafer_n):
    Lot_wafer_number_to_family = f"""
        with scribeNumber AS (
        select WaferScribe, lot, WaferNumber from main.mes.vtmpfablotwaferdetails 
        where Site = 'DFW'
    ),
    -- gives scribed number count that both have

    scribe_family_in_DFW AS (
        Select distinct(ll.vcFamily) as scribe_family, SN.lot as lot, SN.WaferNumber as WaferNumber, VA.vcDesignator
        FROM main.raw_yield.vsmtdata_all as VA
        join main.raw_yield.vsmt_loadlot as ll
        ON ll.vcLotCode = VA.vcLotCode
        join scribeNumber as SN
        ON SN.WaferScribe = VA.vcWaferID
        where VA.vcWaferID = SN.WaferScribe
    )
    -- gives scribed number count that VA have
    select scribe_family, vcDesignator
    from scribe_family_in_DFW
    Where lot = '{lot}' and  WaferNumber = {wafer_n}
    -- test case HJFHF25008	ff1301E	9
    limit 10
    ;
    """
    return spark.sql(Lot_wafer_number_to_family)


def querying_wafer_data(lot, wafer, family, designator):
  aggregation_query = f'''
  WITH base_query AS (
    SELECT MES.ParentLot, MES.WaferNumber, a.Hardbin, a.{designator}, a.Barcode, a.Softbin
    FROM main.testdw.{family}_finaltests AS a
    INNER JOIN (
        SELECT ParentLot, WaferScribe, WaferNumber
        FROM main.mes.vfablotwaferdetails
    ) AS MES ON MES.WaferScribe = a.{designator}
    WHERE a.touchID = (
        SELECT max(touchID)
        FROM main.testdw.{family}_finaltests
        WHERE Barcode = a.Barcode
    )
    AND mes.ParentLot = '{lot}'
    AND mes.WaferNumber in ({wafer})
),
pivoted_data AS (
    SELECT ParentLot, WaferNumber,
    Count(DISTINCT(Softbin)) AS Distinct_Softbin_Values,
           SUM(CASE WHEN Hardbin = 1 THEN 1 ELSE 0 END) AS H_Bin_1,
           SUM(CASE WHEN Hardbin = 2 THEN 1 ELSE 0 END) AS H_Bin_2,
           SUM(CASE WHEN Hardbin = 7 THEN 1 ELSE 0 END) AS H_Bin_7
           -- SUM(CASE WHEN Softbin = 1 THEN 1 ELSE 0 END) AS S_Bin_1,
           -- SUM(CASE WHEN Softbin = 2 THEN 1 ELSE 0 END) AS S_Bin_2,
           -- SUM(CASE WHEN Softbin = 7 THEN 1 ELSE 0 END) AS S_Bin_7
    FROM base_query
    GROUP BY ParentLot, WaferNumber
)
SELECT ParentLot, WaferNumber, H_Bin_1, H_Bin_2, H_Bin_7, Distinct_Softbin_Values, Round(H_Bin_1/(H_Bin_1+H_Bin_2), 4) as Yield, Round(1-Yield, 4) as Yield_Loss
FROM pivoted_data;'''

  return spark.sql(aggregation_query)



# Main function for generating output
def lot_wafer_to_bin_finder(dicts_of_lots_and_wafer = None, output_path = None, input_path = None):
  keep = True
  table_list = []
  while (keep):
    try:
      if dicts_of_lots_and_wafer:
        for key, value in dicts_of_lots_and_wafer.items():
          print(key, value)
          for i in range(len(value)):
              lot = key
              wafer = value[i]
              first_query = Lot_wafer_to_family(lot, wafer).collect()
              scribe_family = first_query[0][0].lower()
              designator = first_query[0][1]
              #   designator = first_query.select('vcDesignator').collect()[0][0]
              table = querying_wafer_data(lot, wafer, scribe_family, designator)
              print(f'The lot number and wafer number belongs to {scribe_family}, and the bin is in the following:')
              if output_path:
                df = table.toPandas()
                df.to_csv(f"{output_path+lot+'-'+str(wafer)+str(i)}.csv")
              table.show()
      elif input_path:
        res_df = pd.DataFrame()
        for key, value in eng_dic(input_path).items():
          print(key, value)
          for i in range(len(value)):
              lot = key
              wafer = value[i]
              first_query = Lot_wafer_to_family(lot, wafer).collect()
              scribe_family = first_query[0][0].lower()
              designator = first_query[0][1]
              #   designator = first_query.select('vcDesignator').collect()[0][0]
              table = querying_wafer_data(lot, wafer, scribe_family, designator)
              print(f'The lot number and wafer number belongs to {scribe_family}, and the bin is in the following:')
              table.show()
              if output_path:
                df = table.toPandas()
                res_df = pd.concat([res_df, df], ignore_index=True)
        res_df.to_csv(f"{output_path}combined.csv")
               
      else:
        lot = str(input('Please enter the lot number: '))
        wafer = str(input('Please enter the wafer number: '))
    #   EG = str(input('Please enter the EG(MesProduct for example: EG-M09558): '))
        first_query = Lot_wafer_to_family(lot, wafer).collect()
        scribe_family = first_query[0][0].lower()
        designator = first_query[0][1]
      #   designator = first_query.select('vcDesignator').collect()[0][0]
        table = querying_wafer_data(lot, wafer, scribe_family, designator)
        print(f'The lot number and wafer number belongs to {scribe_family}, and the bin is in the following:')
        table.show()
        if output_path:
          df = table.toPandas()
          # display(df.select("*"))
          df.to_csv(f"{output_path+lot+'-'+str(wafer)}.csv")
      stay = int(input('If you would like to stop querying enter 0 \nIf you would like to continue querying enter 1: '))
      if stay == 0:
        keep = False
    except Exception as e:
      print(f"There is no data associated with lot {lot} and wafer {wafer} or no finaltest table for family {scribe_family} or you might already have the output CSV. Please try again.")
      stay = int(input('If you would like to stop querying enter 0 \nIf you would like to continue querying enter 1: '))
      if stay == 0:
        keep = False

#1:  Single lot-wafer

Syntax: 
- lot_wafer_to_bin_finder()                                     when you only want to see the output
- lot_wafer_to_bin_finder(output_path = /path/to/your/folder)   when you want to have csv as your output


#2:  List of lots-wafers
lots_and_wafer = {  
  2403242: [1,2,3]   
  2418766: [1,5,7]     
}

Syntax: 
- lot_wafer_to_bin_finder(lots_and_wafer) when you only want to see the output
- lot_wafer_to_bin_finder(lots_and_wafer, output_path = '/path/to/your/folder') when you don't want input csv and want multiple output csv
- lot_wafer_to_bin_finder( input_path = 'path/to/your/csv/file', output_path = '/path/to/your/folder') when you want to have 1 csv as your input and output

#### Note: the input CSV must have lot at the first column and wafer number at the second column

#3:  Single Lot  (analyze all wafers in lot)
syntax: 
- lot_to_bin_finder() when you only want to see the output
- lot_to_bin_finder(output_path = 'your/path') when you want a output csv

In [0]:
import pandas as pd

def extract_c1(path):
  df = pd.read_csv(path)
  df= list(df.iloc[: ,0])
  return df

#functions for task 3
def Lot_to_family(lot):
    Lot_to_family = f"""
        with scribeNumber AS (
        select WaferScribe, lot, WaferNumber from main.mes.vtmpfablotwaferdetails 
        where Site = 'DFW'
    ),
    -- gives scribed number count that both have

    scribe_family_in_DFW AS (
        Select distinct(ll.vcFamily) as scribe_family, SN.lot as lot, SN.WaferNumber as WaferNumber, VA.vcDesignator
        FROM main.raw_yield.vsmtdata_all as VA
        join main.raw_yield.vsmt_loadlot as ll
        ON ll.vcLotCode = VA.vcLotCode
        join scribeNumber as SN
        ON SN.WaferScribe = VA.vcWaferID
        where VA.vcWaferID = SN.WaferScribe
    )
    -- gives scribed number count that VA have
    select scribe_family, vcDesignator, WaferNumber
    from scribe_family_in_DFW
    Where lot = '{lot}'
    -- test case QQ654666	2211301E	9
    limit 10
    ;
    """
    return spark.sql(Lot_to_family)


def querying_wafer_data(lot, wafer, family, designator):
  aggregation_query = f'''
  WITH base_query AS (
    SELECT MES.ParentLot, MES.WaferNumber, a.Hardbin, a.{designator}, a.Barcode, a.Softbin
    FROM main.testdw.{family}_finaltests AS a
    INNER JOIN (
        SELECT ParentLot, WaferScribe, WaferNumber
        FROM main.mes.vfablotwaferdetails
    ) AS MES ON MES.WaferScribe = a.{designator}
    WHERE a.touchID = (
        SELECT max(touchID)
        FROM main.testdw.{family}_finaltests
        WHERE Barcode = a.Barcode
    )
    AND mes.ParentLot = '{lot}'
    AND mes.WaferNumber in ({wafer})
),
pivoted_data AS (
    SELECT ParentLot, WaferNumber,
    Count(DISTINCT(Softbin)) AS Distinct_Softbin_Values,
           SUM(CASE WHEN Hardbin = 1 THEN 1 ELSE 0 END) AS H_Bin_1,
           SUM(CASE WHEN Hardbin = 2 THEN 1 ELSE 0 END) AS H_Bin_2,
           SUM(CASE WHEN Hardbin = 7 THEN 1 ELSE 0 END) AS H_Bin_7
    FROM base_query
    GROUP BY ParentLot, WaferNumber
)
SELECT ParentLot, WaferNumber, H_Bin_1, H_Bin_2, H_Bin_7, Distinct_Softbin_Values, Round(H_Bin_1/(H_Bin_1+H_Bin_2), 4) as Yield, Round(1-Yield, 4) as Yield_Loss
FROM pivoted_data;'''

  return spark.sql(aggregation_query)

  # Main function for generating output
def lot_to_bin_finder(lots = None, input_path = None, output_path=None):
  keep = True
  table_list = []
  if input_path:
    lots = extract_c1(input_path)
  while (keep):
    try:
      if lots:
        res_df = pd.DataFrame() 
        for lot in lots:
          first_query = Lot_to_family(lot).collect()
          scribe_family = first_query[0][0].lower()
          designator = first_query[0][1]
          list_of_wafernumber = []
          for i in range(len(first_query)):
            list_of_wafernumber.append(first_query[i][2])
          string_wafer_ns = [str(wafern) for wafern in list_of_wafernumber]
          result_wafer = ", ".join(string_wafer_ns)
          #   designator = first_query.select('vcDesignator').collect()[0][0]
          table = querying_wafer_data(lot, result_wafer, scribe_family, designator)
          print(f'The lot number and wafer number belongs to {scribe_family}, and the bin is in the following:')
          table.show()
          if output_path:
            df = table.toPandas()
            res_df = pd.concat([res_df, df], ignore_index=True)
        res_df.to_csv(f"{output_path}combined_lot_bin.csv")
          
      else:
        lot = str(input('Please enter the lot number: '))
        first_query = Lot_to_family(lot).collect()
        scribe_family = first_query[0][0].lower()
        designator = first_query[0][1]
        list_of_wafernumber = []
        for i in range(len(first_query)):
          list_of_wafernumber.append(first_query[i][2])
        string_wafer_ns = [str(wafern) for wafern in list_of_wafernumber]
        result_wafer = ", ".join(string_wafer_ns)
        #   designator = first_query.select('vcDesignator').collect()[0][0]
        table = querying_wafer_data(lot, result_wafer, scribe_family, designator)
        print(f'The lot number and wafer number belongs to {scribe_family}, and the bin is in the following:')
        table.show()
        if output_path:
          df = table.toPandas()
          df.to_csv(f"{output_path+lot}.csv")
      stay = int(input('If you would like to stop querying enter 0 \nIf you would like to continue querying enter 1: '))
      if stay == 0:
        keep = False
    except Exception as e:
      print(f"There is no data associated with lot {lot} or no finaltest table for family {scribe_family}. Please try again.")
      lots = lots[1:]
      stay = int(input('If you would like to stop querying enter 0 \nIf you would like to continue querying enter 1: '))
      if stay == 0:
        keep = False


In [0]:
# lot_to_bin_finder() # for just output
lot_to_bin_finder(output_path='/Volumes/scratch/walt_wu/jason/') # for output CSV

# example output:
# Please enter the lot number:
# 2129838
# The lot number and wafer number belongs to qm77180, and the bin is in the following:
# +---------+-----------+-------+-------+-------+-----------------------+------------------+--------------------+
# |ParentLot|WaferNumber|H_Bin_1|H_Bin_2|H_Bin_7|Distinct_Softbin_Values|             Yield|          Yield_Loss|
# +---------+-----------+-------+-------+-------+-----------------------+------------------+--------------------+
# |       1|          6|  52483|   4361|      0|                      2|0.9232812609950038| 0.07671873900499615|
# |       2|          2|  34044|   5566|      0|                      2|0.8594799293107801|  0.1405200706892199|
# |       3|          5|  16297|    848|      0|                     38|0.9505395158938466|0.049460484106153446|
# |       4|          4|  46222|   2421|      0|                     41|0.9502292210595563| 0.04977077894044368|
# |       5|          3|  22772|   2342|      0|                     57|0.9067452416978578| 0.09325475830214225|
# +---------+-----------+-------+-------+-------+-----------------------+------------------+--------------------+



Please enter the lot number:  

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:103)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$2(SequenceExecutionState.scala:103)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$2$adapted(SequenceExecutionState.scala:100)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:100)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:720)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:439)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:439)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecutio

#4:  Multiple Lots  (analyze all wafers in all lots)

syntax:  

-   lots = ['2129838', '2129839']  
lot_to_bin_finder(lots)
Do the above when you specified lots on databrick
- lots = ['2129838', '2129839']  
lot_to_bin_finder(lots, output_path = 'path/to/output/folder/') when you want to save result as an csv
- lot_to_bin_finder(input_path = 'path/to/your/csv/file', output_path = '/path/to/your/folder/') when you want to have 1 csv as your input and output

In [0]:
lots = ['2129838', '2129838']
# lot_to_bin_finder(lots) #for output only
lot_to_bin_finder(input_path='/Volumes/scratch/walt_wu/jason/2129838.csv', output_path='/Volumes/scratch/walt_wu/jason/') 

# example:
# The lot number and wafer number belongs to qm77180, and the bin is in the following:
# +---------+-----------+-------+-------+-------+-------+-------+-------+------------------+--------------------+
# |ParentLot|WaferNumber|H_Bin_1|H_Bin_2|H_Bin_7|S_Bin_1|S_Bin_2|S_Bin_7|             Yield|          Yield_Loss|
# +---------+-----------+-------+-------+-------+-------+-------+-------+------------------+--------------------+
# |  1234567|          6|  52483|   4361|      0|  52483|      0|      0|0.9232812609950038| 0.07671873900499615|
# |  1234567|          2|  34044|   5566|      0|  34044|      0|      0|0.8594799293107801|  0.1405200706892199|
# |  1234567|          5|  16297|    848|      0|  16297|      0|      0|0.9505395158938466|0.049460484106153446|
# |  1234567|          4|  46222|   2421|      0|  46222|      0|      0|0.9502292210595563| 0.04977077894044368|
# |  1234567|          3|  22772|   2342|      0|  22772|      0|      0|0.9067452416978578| 0.09325475830214225|
# +---------+-----------+-------+-------+-------+-------+-------+-------+------------------+--------------------+


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:438)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecution(ChauffeurState.scala:1297)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:1023)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:532)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:636)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:654)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:48)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:253)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala

# 5. EG to find lot wafer number and Bin
syntax:  
- EG_to_bin_finder()  When you only want the output
- EGs = ['22222', '33333', '44444']  
EG_to_bin_finder(EGs)  when you want to run multiple EGs and output as csv
- EG_to_bin_finder(input='input/path', output_path= 'output/path/)  
when you want input and output both as csv file

In [0]:
from collections import defaultdict
import pandas as pd

def extract_c1(path):
  df = pd.read_csv(path)
  df= list(df.iloc[: ,0])
  return df


#functions for task 5
def EG_to_family(EG):
    Lot_to_family = f"""
        with scribeNumber AS (
        select WaferScribe, lot, WaferNumber
        from main.mes.vtmpfablotwaferdetails 
        where Site = 'DFW'
    ),
    -- gives scribed number count that both have

    scribe_family_in_DFW AS (
        Select distinct(ll.vcFamily) as scribe_family, SN.lot as lot, SN.WaferNumber as WaferNumber, VA.vcDesignator, VA.vcWaferMaterial as EG
        FROM main.raw_yield.vsmtdata_all as VA
        join main.raw_yield.vsmt_loadlot as ll
        ON ll.vcLotCode = VA.vcLotCode
        join scribeNumber as SN
        ON SN.WaferScribe = VA.vcWaferID
        where VA.vcWaferID = SN.WaferScribe
    )
    -- gives scribed number count that VA have
    select scribe_family, vcDesignator, WaferNumber, EG, lot
    from scribe_family_in_DFW
    where EG = '{EG}'
    -- test case QM25008	2211301E	9
    ;
    """
    return spark.sql(Lot_to_family)


def EG_querying_wafer_data(family, designator, EG):
  aggregation_query = f'''
  WITH base_query AS (
    SELECT MES.ParentLot, MES.WaferNumber, a.Hardbin, a.{designator}, a.Barcode, a.Softbin
    FROM main.testdw.{family}_finaltests AS a
    INNER JOIN (
        SELECT ParentLot, WaferScribe, WaferNumber
        FROM main.mes.vfablotwaferdetails
    ) AS MES ON MES.WaferScribe = a.{designator}
    WHERE a.touchID = (
        SELECT max(touchID)
        FROM main.testdw.{family}_finaltests
        WHERE Barcode = a.Barcode
    )
),
pivoted_data AS (
    SELECT ParentLot, WaferNumber,
    Count(DISTINCT(Softbin)) AS Distinct_Softbin_Values,
           SUM(CASE WHEN Hardbin = 1 THEN 1 ELSE 0 END) AS H_Bin_1,
           SUM(CASE WHEN Hardbin = 2 THEN 1 ELSE 0 END) AS H_Bin_2,
           SUM(CASE WHEN Hardbin = 7 THEN 1 ELSE 0 END) AS H_Bin_7
    FROM base_query
    GROUP BY ParentLot, WaferNumber
)
SELECT ParentLot, WaferNumber, H_Bin_1, H_Bin_2, H_Bin_7, Distinct_Softbin_Values, Round(H_Bin_1/(H_Bin_1+H_Bin_2), 4) as Yield, Round(1-Yield, 4) as Yield_Loss
FROM pivoted_data;
;'''

  return spark.sql(aggregation_query)



# Main function for generating output
def EG_to_bin_finder(EGs = None, input_path = None, output_path=None):
  keep = True
  table_list = []
  if input_path:
    EGs = extract_c1(input_path)
  while (keep):
    try:
      if EGs:
        res_df = pd.DataFrame()
        for EG in EGs:
          first_query = EG_to_family(EG).collect()
        family_des = defaultdict(set)
        for i in range(len(first_query)):
          if first_query[i][0]:
            if first_query[i][0].lower() in family_des:
              family_des[first_query[i][0].lower()].add(first_query[i][1])
            else:
              family_des[first_query[i][0].lower()] = {first_query[i][1]}
          else:
            continue
      for key, value in family_des.items():
        for designator in list(value):
          print(key, designator, EG)
          table = EG_querying_wafer_data(key, designator, EG)
          print(f'The EG query belongs to family {key} and designator {designator}, and the bin is in the following:')
          table.show()
          if output_path:
            df = table.toPandas()
            res_df = pd.concat([res_df, df], ignore_index=True)
        res_df.to_csv(f"{output_path}combined_EG_bin.csv")


      else:
        EG = str(input('Please enter the EG(MesProduct for example: EG-M09558): '))
        family_des = defaultdict(set)
        first_query = EG_to_family(EG).collect()
        family_des = defaultdict(set)
        for i in range(len(first_query)):
          if first_query[i][0]:
            if first_query[i][0].lower() in family_des:
              family_des[first_query[i][0].lower()].add(first_query[i][1])
            else:
              family_des[first_query[i][0].lower()] = {first_query[i][1]}
          else:
            continue
      for key, value in family_des.items():
        for designator in list(value):
          print(key, designator, EG)
          table = EG_querying_wafer_data(key, designator, EG)
          print(f'The EG query belongs to family {key} and designator {designator}, and the bin is in the following:')
          table.show()
          
      stay = int(input('If you would like to stop querying enter 0 \nIf you would like to continue querying enter 1: '))
      if stay == 0:
        keep = False
    except Exception as e:
      print(f"There is no data associated with EG {EG}. Please try again.")
      stay = int(input('If you would like to stop querying enter 0 \nIf you would like to continue querying enter 1: '))
      if stay == 0:
        keep = False

# V.2 functions

## Having some updates on V1 query result and ability to download query result into CSVs  
- Query output no longer have softbin 1,2,7 counts, instead adding count of distinct softbins
- Rounding yield and yield loss to the 4th decimals
- Functions new ability to enter the path and download query result into CSVs

#### Note: all the V2 functions' syntax is the same as V1

In [0]:
import pandas as pd
from collections import defaultdict
# Read the first CSV file and data engineering
def eng_dic(input_path):
  res = defaultdict(list)
  df = pd.read_csv(input_path)
  df.rename(columns={df.columns[0]: 'Lot', df.columns[1]: 'Wafer'}, inplace=True)
  for i in range(len(df)):
    res[str(df.loc[(i), 'Lot'])].append(df.loc[(i), 'Wafer']) 
  return res

# sub function for query
def Lot_wafer_to_family_v2(lot, wafer_n):
    Lot_wafer_number_to_family = f"""
        with scribeNumber AS (
        select WaferScribe, lot, WaferNumber from main.mes.vtmpfablotwaferdetails 
        where Site = 'DFW'
    ),
    -- gives scribed number count that both have

    scribe_family_in_DFW AS (
        Select distinct(ll.vcFamily) as scribe_family, SN.lot as lot, SN.WaferNumber as WaferNumber, VA.vcDesignator, VA.vcWaferMaterial as EG
        FROM main.raw_yield.vsmtdata_all as VA
        join main.raw_yield.vsmt_loadlot as ll
        ON ll.vcLotCode = VA.vcLotCode
        join scribeNumber as SN
        ON SN.WaferScribe = VA.vcWaferID
        where VA.vcWaferID = SN.WaferScribe
    )
    -- gives scribed number count that VA have
    select scribe_family, vcDesignator, EG
    from scribe_family_in_DFW
    Where lot = '{lot}' and  WaferNumber = {wafer_n};
    -- test case QM25008	2211301E	9
    """
    return spark.sql(Lot_wafer_number_to_family)


def querying_wafer_data_v2(lot, wafer, family, designator, EG):
  aggregation_query = f'''
  WITH base_query AS (
    SELECT MES.ParentLot, MES.WaferNumber, a.Hardbin, a.{designator}, a.Barcode, a.Softbin
    FROM main.testdw.{family}_finaltests AS a
    INNER JOIN (
        SELECT ParentLot, WaferScribe, WaferNumber
        FROM main.mes.vfablotwaferdetails
    ) AS MES ON MES.WaferScribe = a.{designator}
    WHERE a.touchID = (
        SELECT max(touchID)
        FROM main.testdw.{family}_finaltests
        WHERE Barcode = a.Barcode
    )
    AND mes.ParentLot = '{lot}'
    AND mes.WaferNumber in ({wafer})
),
pivoted_data AS (
    SELECT 
    '{family}' as Family,
    '{EG}' as EG,
    ParentLot, 
    WaferNumber,
    Hardbin as HardBin_Type,
    Softbin as SoftBin_Type,
    Count(*) AS Module_Count
    FROM base_query
    GROUP BY ParentLot, WaferNumber, Hardbin, Softbin
)
SELECT Family, EG, ParentLot, WaferNumber, HardBin_Type, SoftBin_Type, Module_Count
FROM pivoted_data;'''

  return spark.sql(aggregation_query)



# Main function for generating output
def lot_wafer_to_bin_finder_v2(dicts_of_lots_and_wafer = None, output_path = None, input_path = None):
  keep = True
  table_list = []
  if input_path:
    dicts_of_lots_and_wafer = eng_dic(input_path)
  while (keep):
    # try:
    if dicts_of_lots_and_wafer:
      for key, value in dicts_of_lots_and_wafer.items():
        print(key, value)
        for i in range(len(value)):
            lot = key
            wafer = value[i]
            first_query = Lot_wafer_to_family_v2(lot, wafer).collect()
            scribe_family = first_query[0][0].lower()
            designator = first_query[0][1]
            EG = first_query[0][2]
            #   designator = first_query.select('vcDesignator').collect()[0][0]
            table = querying_wafer_data_v2(lot, wafer, scribe_family, designator, EG)
            print(f'The lot number and wafer number belongs to {scribe_family}, and the bin is in the following:')
            if output_path:
              df = table.toPandas()
              df.to_csv(f"{output_path+lot+'-'+str(wafer)+str(i)}.csv")
            table.show()
    else:
      lot = str(input('Please enter the lot number: '))
      wafer = str(input('Please enter the wafer number: '))
  #   EG = str(input('Please enter the EG(MesProduct for example: EG-M09558): '))
      first_query = Lot_wafer_to_family_v2(lot, wafer).collect()
      scribe_family = first_query[0][0].lower()
      designator = first_query[0][1]
      EG = first_query[0][2]
      #   designator = first_query.select('vcDesignator').collect()[0][0]
      table = querying_wafer_data_v2(lot, wafer, scribe_family, designator, EG)
      print(f'The lot number and wafer number belongs to {scribe_family}, and the bin is in the following:')
      table.show()
      if output_path:
        df = table.toPandas()
        # display(df.select("*"))
        df.to_csv(f"{output_path+lot+'-'+str(wafer)}.csv")
    stay = int(input('If you would like to stop querying enter 0 \nIf you would like to continue querying enter 1: '))
    if stay == 0:
      keep = False
    # except Exception as e:
    #   print(f"There is no data associated with lot {lot} and wafer {wafer} or no finaltest table for family {scribe_family} or you might already have the output CSV. Please try again.")
    #   stay = int(input('If you would like to stop querying enter 0 \nIf you would like to continue querying enter 1: '))
    #   if stay == 0:
    #     keep = False

%md
#1:  Single lot-wafer
#2:  Input list of lot and wafers (Through direct input and CSV)



# 3: Single lot 
# 4: Multiple Lots 

In [0]:
import pandas as pd

def extract_c1(path):
  df = pd.read_csv(path)
  df= list(df.iloc[: ,0])
  return df

#functions for task 3
def Lot_to_family_v2(lot):
    Lot_to_family = f"""
        with scribeNumber AS (
        select WaferScribe, lot, WaferNumber from main.mes.vtmpfablotwaferdetails 
        where Site = 'DFW'
    ),
    -- gives scribed number count that both have

    scribe_family_in_DFW AS (
        Select distinct(ll.vcFamily) as scribe_family, SN.lot as lot, SN.WaferNumber as WaferNumber, VA.vcDesignator, VA.vcWaferMaterial as EG
        FROM main.raw_yield.vsmtdata_all as VA
        join main.raw_yield.vsmt_loadlot as ll
        ON ll.vcLotCode = VA.vcLotCode
        join scribeNumber as SN
        ON SN.WaferScribe = VA.vcWaferID
        where VA.vcWaferID = SN.WaferScribe
    )
    -- gives scribed number count that VA have
    select scribe_family, vcDesignator, WaferNumber, EG
    from scribe_family_in_DFW
    Where lot = '{lot}'
    -- test case QM25008	2211301E	9
    limit 10
    ;
    """
    return spark.sql(Lot_to_family)


def querying_wafer_data_v2(lot, wafer, family, designator, EG):
  aggregation_query = f'''
  WITH base_query AS (
    SELECT MES.ParentLot, MES.WaferNumber, a.Hardbin, a.{designator}, a.Barcode, a.Softbin
    FROM main.testdw.{family}_finaltests AS a
    INNER JOIN (
        SELECT ParentLot, WaferScribe, WaferNumber
        FROM main.mes.vfablotwaferdetails
    ) AS MES ON MES.WaferScribe = a.{designator}
    WHERE a.touchID = (
        SELECT max(touchID)
        FROM main.testdw.{family}_finaltests
        WHERE Barcode = a.Barcode
    )
    AND mes.ParentLot = '{lot}'
    AND mes.WaferNumber in ({wafer})
),
pivoted_data AS (
    SELECT 
    '{family}' as Family,
    '{EG}' as EG,
    ParentLot, 
    WaferNumber,
    Hardbin as HardBin_Type,
    Softbin as SoftBin_Type,
    Count(*) AS Module_Count
    FROM base_query
    GROUP BY ParentLot, WaferNumber, Hardbin, Softbin
)
SELECT Family, EG, ParentLot, WaferNumber, HardBin_Type, SoftBin_Type, Module_Count
FROM pivoted_data;'''

  return spark.sql(aggregation_query)

  # Main function for generating output
def lot_to_bin_finder_v2(lots = None, input_path = None, output_path=None):
  keep = True
  table_list = []
  if input_path:
    lots = extract_c1(input_path)
  while (keep):
    # try:
    if lots:
      res_df = pd.DataFrame() 
      for lot in lots:
        first_query = Lot_to_family_v2(lot).collect()
        scribe_family = first_query[0][0].lower()
        designator = first_query[0][1]
        EG = first_query[0][3]
        print(EG)
        list_of_wafernumber = []
        for i in range(len(first_query)):
          list_of_wafernumber.append(first_query[i][2])
        string_wafer_ns = [str(wafern) for wafern in list_of_wafernumber]
        result_wafer = ", ".join(string_wafer_ns)
        #   designator = first_query.select('vcDesignator').collect()[0][0]
        table = querying_wafer_data_v2(lot, result_wafer, scribe_family, designator, EG)
        print(f'The lot number and wafer number belongs to {scribe_family}, and the bin is in the following:')
        table.show()
        if output_path:
          df = table.toPandas()
          res_df = pd.concat([res_df, df], ignore_index=True)
      res_df.to_csv(f"{output_path}combined_lot_bin.csv")
        
    else:
      lot = str(input('Please enter the lot number: '))
      first_query = Lot_to_family_v2(lot).collect()
      scribe_family = first_query[0][0].lower()
      designator = first_query[0][1]
      EG = first_query[0][3]
      list_of_wafernumber = []
      for i in range(len(first_query)):
        list_of_wafernumber.append(first_query[i][2])
      string_wafer_ns = [str(wafern) for wafern in list_of_wafernumber]
      result_wafer = ", ".join(string_wafer_ns)
      #   designator = first_query.select('vcDesignator').collect()[0][0]
      table = querying_wafer_data_v2(lot, result_wafer, scribe_family, designator, EG)
      print(f'The lot number and wafer number belongs to {scribe_family}, and the bin is in the following:')
      table.show()
      if output_path:
        df = table.toPandas()
        df.to_csv(f"{output_path+lot}_bin.csv")
    stay = int(input('If you would like to stop querying enter 0 \nIf you would like to continue querying enter 1: '))
    if stay == 0:
      keep = False
    # except Exception as e:
    #   print(f"There is no data associated with lot {lot} or no finaltest table for family {scribe_family}. Please try again.")
    #   lots = lots[1:]
    #   stay = int(input('If you would like to stop querying enter 0 \nIf you would like to continue querying enter 1: '))
    #   if stay == 0:
    #     keep = False


# 5: EG

In [0]:
from collections import defaultdict
import pandas as pd

def extract_c1(path):
  df = pd.read_csv(path)
  df= list(df.iloc[: ,0])
  return df


#functions for task 5
def EG_to_family_v2(EG):
    Lot_to_family = f"""
        with scribeNumber AS (
        select WaferScribe, lot, WaferNumber
        from main.mes.vtmpfablotwaferdetails 
        where Site = 'DFW'
    ),
    -- gives scribed number count that both have

    scribe_family_in_DFW AS (
        Select distinct(ll.vcFamily) as scribe_family, SN.lot as lot, SN.WaferNumber as WaferNumber, VA.vcDesignator, VA.vcWaferMaterial as EG
        FROM main.raw_yield.vsmtdata_all as VA
        join main.raw_yield.vsmt_loadlot as ll
        ON ll.vcLotCode = VA.vcLotCode
        join scribeNumber as SN
        ON SN.WaferScribe = VA.vcWaferID
        where VA.vcWaferID = SN.WaferScribe
    )
    -- gives scribed number count that VA have
    select scribe_family, vcDesignator, WaferNumber, EG, lot
    from scribe_family_in_DFW
    where EG = '{EG}'
    -- test case QM25008	2211301E	9
    ;
    """
    return spark.sql(Lot_to_family)


def EG_querying_wafer_data_v2(family, designator, EG):
  aggregation_query = f'''
  WITH base_query AS (
    SELECT MES.ParentLot, MES.WaferNumber, a.Hardbin, a.{designator}, a.Barcode, a.Softbin
    FROM main.testdw.{family}_finaltests AS a
    INNER JOIN (
        SELECT ParentLot, WaferScribe, WaferNumber
        FROM main.mes.vfablotwaferdetails
    ) AS MES ON MES.WaferScribe = a.{designator}
    WHERE a.touchID = (
        SELECT max(touchID)
        FROM main.testdw.{family}_finaltests
        WHERE Barcode = a.Barcode
    )
),
pivoted_data AS (
    SELECT 
    '{family}' as Family,
    '{EG}' as EG,
    ParentLot, 
    WaferNumber,
    Hardbin as HardBin_Type,
    Softbin as SoftBin_Type,
    Count(*) AS Module_Count
    FROM base_query
    GROUP BY ParentLot, WaferNumber, Hardbin, Softbin
)
SELECT Family, EG, ParentLot, WaferNumber, HardBin_Type, SoftBin_Type, Module_Count
FROM pivoted_data;'''

  return spark.sql(aggregation_query)



# Main function for generating output
def EG_to_bin_finder_v2(EGs = None, input_path = None, output_path=None):
  keep = True
  table_list = []
  if input_path:
    EGs = extract_c1(input_path)
  while (keep):
    try:
      if EGs:
        res_df = pd.DataFrame()
        for EG in EGs:
          first_query = EG_to_family_v2(EG).collect()
        family_des = defaultdict(set)
        for i in range(len(first_query)):
          if first_query[i][0]:
            if first_query[i][0].lower() in family_des:
              family_des[first_query[i][0].lower()].add(first_query[i][1])
            else:
              family_des[first_query[i][0].lower()] = {first_query[i][1]}
          else:
            continue
      for key, value in family_des.items():
        for designator in list(value):
          print(key, designator, EG)
          table = EG_querying_wafer_data_v2(key, designator, EG)
          print(f'The EG query belongs to family {key} and designator {designator}, and the bin is in the following:')
          table.show()
          if output_path:
            df = table.toPandas()
            res_df = pd.concat([res_df, df], ignore_index=True)
        res_df.to_csv(f"{output_path}combined_EG_bin.csv")


      else:
        EG = str(input('Please enter the EG(MesProduct for example: EG-M09558): '))
        family_des = defaultdict(set)
        first_query = EG_to_family_v2(EG).collect()
        family_des = defaultdict(set)
        for i in range(len(first_query)):
          if first_query[i][0]:
            if first_query[i][0].lower() in family_des:
              family_des[first_query[i][0].lower()].add(first_query[i][1])
            else:
              family_des[first_query[i][0].lower()] = {first_query[i][1]}
          else:
            continue
      for key, value in family_des.items():
        for designator in list(value):
          print(key, designator, EG)
          table = EG_querying_wafer_data_v2(key, designator, EG)
          print(f'The EG query belongs to family {key} and designator {designator}, and the bin is in the following:')
          table.show()
          
      stay = int(input('If you would like to stop querying enter 0 \nIf you would like to continue querying enter 1: '))
      if stay == 0:
        keep = False
    except Exception as e:
      print(f"There is no data associated with EG {EG}. Please try again.")
      stay = int(input('If you would like to stop querying enter 0 \nIf you would like to continue querying enter 1: '))
      if stay == 0:
        keep = False

# Some notes for myself:

In [0]:

aggregation_query_test = '''
WITH base_query AS (
    SELECT MES.ParentLot, MES.WaferNumber, a.Hardbin, a.U10, a.Barcode, a.Softbin
    FROM main.testdw.qm77180_finaltests AS a
    INNER JOIN (
        SELECT ParentLot, WaferScribe, WaferNumber
        FROM main.mes.vfablotwaferdetails
    ) AS MES ON MES.WaferScribe = a.U14
    WHERE a.touchID = (
        SELECT max(touchID)
        FROM main.testdw.qm77180_finaltests
        WHERE Barcode = a.Barcode
    )
),
pivoted_data AS (
    SELECT ParentLot, WaferNumber,
    Count(DISTINCT(Softbin)) AS Distinct_Softbin_Values,
           SUM(CASE WHEN Hardbin = 1 THEN 1 ELSE 0 END) AS H_Bin_1,
           SUM(CASE WHEN Hardbin = 2 THEN 1 ELSE 0 END) AS H_Bin_2,
           SUM(CASE WHEN Hardbin = 7 THEN 1 ELSE 0 END) AS H_Bin_7
           -- SUM(CASE WHEN Softbin = 1 THEN 1 ELSE 0 END) AS S_Bin_1,
           -- SUM(CASE WHEN Softbin = 2 THEN 1 ELSE 0 END) AS S_Bin_2,
           -- SUM(CASE WHEN Softbin = 7 THEN 1 ELSE 0 END) AS S_Bin_7
    FROM base_query
    GROUP BY ParentLot, WaferNumber
)
SELECT ParentLot, WaferNumber, H_Bin_1, H_Bin_2, H_Bin_7, Distinct_Softbin_Values, Round(H_Bin_1/(H_Bin_1+H_Bin_2), 4) as Yield, Round(1-Yield, 4) as Yield_Loss
FROM pivoted_data;
'''


df = spark.sql(aggregation_query_test)

In [0]:
import pandas as pd

eng_dic('/Volumes/scratch/walt_wu/jason/Databricks Module Report mockups  2024_0604a.csv')


In [0]:
df

In [0]:
%sql
SELECT *
  FROM (
      SELECT MES.ParentLot, MES.WaferNumber, a.Hardbin, a.FL3439, a.Barcode
      FROM main.testdw.qm77298_finaltests AS a
      INNER JOIN (
          SELECT ParentLot, WaferScribe, WaferNumber
          FROM main.mes.vfablotwaferdetails
      ) AS MES ON MES.WaferScribe = a.FL3439
      WHERE a.touchID = (
          SELECT max(touchID)
          FROM main.testdw.qm77298_finaltests
          WHERE Barcode = a.Barcode
      )
          AND mes.ParentLot = '2324406'
        AND mes.WaferNumber = 1
  ) subquery
  PIVOT (
      COUNT(Barcode)
      FOR Hardbin IN (1, 2, 7)
  )

In [0]:

def querying_family_data(lot):
  find_family = f'''Select Distinct(LL.vcFamily)
from main.raw_yield.vsmt_loadlot LL
join main.raw_yield.vsmtdata_all A
on LL.vcLotCode = A.vcLotCode
WHERE LL.vcLotCode = '{lot}'
limit 10;'''
  return spark.sql(find_family)

def family_to_data(family):
  aggregation_query = f'''SELECT *
  FROM (
      SELECT MES.ParentLot, MES.WaferNumber, a.Hardbin, a.FL3439, a.Barcode
      FROM main.testdw.{family}_finaltests AS a
      INNER JOIN (
          SELECT ParentLot, WaferScribe, WaferNumber
          FROM main.mes.vfablotwaferdetails
      ) AS MES ON MES.WaferScribe = a.FL3439
      WHERE a.touchID = (
          SELECT max(touchID)
          FROM main.testdw.{family}_finaltests
          WHERE Barcode = a.Barcode
      )
        --  AND mes.ParentLot = '{lot}'
        -- AND mes.WaferNumber = {wafer}
  ) subquery
  PIVOT (
      COUNT(Barcode)
      FOR Hardbin IN (1, 2, 7)
  );'''

  return spark.sql(aggregation_query)

def EG_to_bin_finder():
  keep = True
  while (keep):
    try:
      lot = str(input('Please enter the lot number: '))
      family = querying_family_data(lot).select('vcFamily').collect()[0][0]
      table = querying_wafer_data(lot, wafer, EG)
      table.show()
      stay = int(input('If you would like to stop querying enter 0 \nIf you would like to continue querying enter 1: '))
      if stay == 0:
        keep = False
    except Exception as e:
      print(f"There is no data associated with lot {lot}, wafer {wafer}, and EG {EG}. Please try again.")
      stay = int(input('If you would like to stop querying enter 0 \nIf you would like to continue querying enter 1: '))
      if stay == 0:
        keep = False
  

In [0]:
first_query = EG_to_family('WEG9197').collect()

In [0]:
print(first_query[561][0])
family_des = defaultdict(set)
for i in range(len(first_query)):
  if first_query[i][0]:
    if first_query[i][0].lower() in family_des:
      family_des[first_query[i][0].lower()].add(first_query[i][1])
    else:
      family_des[first_query[i][0].lower()] = {first_query[i][1]}
  else:
    continue
print(family_des)
print('here2222')

In [0]:
for key, value in family_des.items():
      for designator in list(value):
        print(key, designator)